In [21]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

import glob, os
glob.glob(os.path.join(os.environ["SPARK_HOME"], "conf", "spark*"))

['/Users/rushi/spark/conf/spark-env.sh.template',
 '/Users/rushi/spark/conf/spark-defaults.conf',
 '/Users/rushi/spark/conf/spark-defaults.conf.template']

In [22]:
#read csv

testdf = pd.read_csv('datasets/test.csv')
traindf = pd.read_csv('datasets/train.csv')
alldf = pd.read_csv('datasets/submit.csv')

print(testdf.shape, traindf.shape)

print('cleaning dataframes')

testdf = testdf.dropna()
traindf = traindf.dropna()

print(testdf.shape, traindf.shape)

ytestdf = testdf.drop(['title', 'author', 'text'], axis=1)

# ytestdf['labels'] = np.zeros((len(testdf), 1))

(5200, 4) (20800, 5)
cleaning dataframes
(4575, 4) (18285, 5)


In [23]:
# ytestdf = ytestdf.merge(alldf, left_on='id', right_on='id', how='left')[['id', 'label']]
ytestdf = pd.merge(left = ytestdf, right=alldf, left_on='id', right_on='id', how='left')

In [24]:
spark = SparkSession.builder.\
    master('local[*]').\
        config('spark.driver.memory', '10g').\
            appName('fakenews').\
                getOrCreate()

print(spark.sparkContext.uiWebUrl)
print(spark.sparkContext.getConf().getAll())
    # master('local[*]').\
    #     config('spark.driver.memory','10g').\
print(spark)

http://d-i89-236-125.student.eduroam.uq.edu.au:4040
[('spark.app.name', 'fakenews'), ('spark.sql.warehouse.dir', 'file:/Users/rushi/Documents/GitHub/fake-news-detection/spark-warehouse'), ('spark.driver.memory', '10g'), ('spark.executor.id', 'driver'), ('spark.app.id', 'local-1648174324863'), ('spark.driver.port', '52212'), ('spark.driver.host', 'd-i89-236-125.student.eduroam.uq.edu.au'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.app.startTime', '1648174324821'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [25]:
from pyspark.sql.types import StructField, StructType
from pyspark.sql.types import IntegerType, StringType, DoubleType

trainschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType()),\
                StructField('label', IntegerType())]) 

testschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType())]) 

trainset = spark.read.csv('datasets/train.csv', schema=trainschema).dropna()
testset = spark.read.csv('datasets/test.csv', schema=testschema).dropna()

print(trainset.count(), testset.count())

trainset.printSchema()
testset.printSchema()

11844 5072
root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- text: string (nullable = true)



In [26]:
trainset.show(10)

testset.show(10)

+---+--------------------+--------------------+--------------------+-----+
| id|               title|              author|                text|label|
+---+--------------------+--------------------+--------------------+-----+
|  1|FLYNN: Hillary Cl...|     Daniel J. Flynn|Ever get the feel...|    0|
|  5|Jackie Mason: Hol...|     Daniel Nussbaum|In these trying t...|    0|
|  7|Benoît Hamon Wins...|     Alissa J. Rubin|PARIS  —   France...|    0|
|  8|Excerpts From a D...|                 nan|Donald J. Trump i...|    0|
|  9|A Back-Channel Pl...|Megan Twohey and ...|A week before Mic...|    0|
| 10|Obama’s Organizin...|         Aaron Klein|Organizing for Ac...|    0|
| 11|"BBC Comedy Sketc...|     Chris Tomlinson|The BBC produced ...|    0|
| 15|In Major League S...|       Jack Williams|Guillermo Barros ...|    0|
| 16|Wells Fargo Chief...|Michael Corkery a...|The scandal engul...|    0|
| 19|Chuck Todd: ’Buzz...|           Jeff Poor|Wednesday after  ...|    0|
+---+--------------------

In [27]:
# xtrain, ytrain = trainset[['id', 'title', 'author', 'text']], trainset[['id', 'label']]

# xtrain.show(10)
# ytrain.show(10)

In [28]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, CountVectorizer, StopWordsRemover
#tonkenize text

tokenizer = Tokenizer(outputCol='words')
tokenizer.setInputCol('text')
wordsdata = tokenizer.transform(trainset)

stopwr = StopWordsRemover(inputCol='words', outputCol='filtered')
wordsdata = stopwr.transform(wordsdata)

hashing = HashingTF(inputCol='filtered', outputCol='featuresHTF')
wordsdata = hashing.transform(wordsdata)

vector = CountVectorizer(inputCol='filtered', outputCol='featuresCV')
model = vector.fit(wordsdata)

wordsdata = model.transform(wordsdata)


wordsdata.show(10)

+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
| id|               title|              author|                text|label|               words|            filtered|         featuresHTF|          featuresCV|
+---+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|  1|FLYNN: Hillary Cl...|     Daniel J. Flynn|Ever get the feel...|    0|[ever, get, the, ...|[ever, get, feeli...|(262144,[796,917,...|(260094,[0,2,3,4,...|
|  5|Jackie Mason: Hol...|     Daniel Nussbaum|In these trying t...|    0|[in, these, tryin...|[trying, times,, ...|(262144,[1689,425...|(260094,[0,2,5,9,...|
|  7|Benoît Hamon Wins...|     Alissa J. Rubin|PARIS  —   France...|    0|[paris, , —, , , ...|[paris, , —, , , ...|(262144,[965,1576...|(260094,[0,1,2,3,...|
|  8|Excerpts From a D...|                 nan

In [29]:
#create a model
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol='featuresHTF', labelCol='label')
gbt.fit(wordsdata)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/rushi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-d033c49b67ad>", line 5, in <module>
    gbt.fit(wordsdata)
  File "/Users/rushi/spark/python/pyspark/ml/base.py", line 161, in fit
    return self._fit(dataset)
  File "/Users/rushi/spark/python/pyspark/ml/wrapper.py", line 335, in _fit
    java_model = self._fit_java(dataset)
  File "/Users/rushi/spark/python/pyspark/ml/wrapper.py", line 332, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/Users/rushi/opt/anaconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/Users/rushi/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/Users/rushi/opt/anaconda3/lib/python3.8/site-packages/py4j/proto

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
spark.stop()